In [2]:
import torch
import pandas as pd
import numpy as np

In [42]:
import torch
import torch.nn as nn

class FootballLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(FootballLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # LSTM for general features
        self.lstm_general = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Separate LSTM for h2h features
        self.lstm_h2h = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Output layers for multi-output predictions
        self.fc_result = nn.Linear(hidden_dim * 2, 3)  # match result (win/lose/draw)
        self.fc_goals = nn.Linear(hidden_dim * 2, 1)  # number of goals
        self.fc_corners = nn.Linear(hidden_dim * 2, 1)  # number of corner kicks
        self.fc_fouls = nn.Linear(hidden_dim * 2, 1)  # number of fouls
        self.fc_cards = nn.Linear(hidden_dim * 2, 1)  # number of cards

    def forward(self, general_features, h2h_features):
        # Processing general features
        _, (ht_general, _) = self.lstm_general(general_features)  # ht_general shape: [num_layers, batch, hidden_dim]
        
        # Processing head-to-head features
        _, (ht_h2h, _) = self.lstm_h2h(h2h_features)  # ht_h2h shape: [num_layers, batch, hidden_dim]
        
        # Concatenating the final hidden states from the last layer of both LSTMs
        # Take the last layer's hidden states (last layer from both LSTMs)
        ht_general_last = ht_general[-1]  # Last layer, shape: [batch, hidden_dim]
        ht_h2h_last = ht_h2h[-1]  # Last layer, shape: [batch, hidden_dim]
        
        # Concatenate along the feature dimension
        combined_ht = torch.cat((ht_general_last, ht_h2h_last), dim=1)  # shape: [batch, 2 * hidden_dim]
        
        # Output calculations
        result = self.fc_result(combined_ht)  # Predicted match result
        goals = self.fc_goals(combined_ht)  # Predicted number of goals
        corners = self.fc_corners(combined_ht)  # Predicted number of corner kicks
        fouls = self.fc_fouls(combined_ht)  # Predicted number of fouls
        cards = self.fc_cards(combined_ht)  # Predicted number of cards
        
        return result, goals, corners, fouls, cards


In [49]:
# Przykładowe dane
batch_size = 1
seq_length = 5
input_dim = 10

general_features = torch.randn(batch_size, seq_length, input_dim)
h2h_features = torch.randn(batch_size, seq_length, input_dim)

# Inicjalizacja modelu
model = FootballLSTM(input_dim=input_dim, hidden_dim=100, num_layers=2)

# Uzyskanie przewidywań
result, goals, corners, fouls, cards = model(general_features, h2h_features)

In [51]:
model(general_features, h2h_features)

(tensor([[ 0.0461, -0.0845, -0.0427]], grad_fn=<AddmmBackward0>),
 tensor([[0.0199]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0095]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0481]], grad_fn=<AddmmBackward0>),
 tensor([[0.0009]], grad_fn=<AddmmBackward0>))

In [52]:
import torch.nn.functional as F

result_probabilities = F.softmax(result, dim=1)
print("Prawdopodobieństwa wyniku meczu:", result_probabilities)

Prawdopodobieństwa wyniku meczu: tensor([[0.3581, 0.3142, 0.3277]], grad_fn=<SoftmaxBackward0>)


In [54]:
0.3581 + 0.3142 + 0.3277

0.9999999999999999

In [55]:
predicted_goals = round(goals.item())
predicted_corners = round(corners.item())
predicted_fouls = round(fouls.item())
predicted_cards = round(cards.item())

print(f"Przewidywana liczba goli: {predicted_goals}")
print(f"Przewidywana liczba rzutów rożnych: {predicted_corners}")
print(f"Przewidywana liczba fauli: {predicted_fouls}")
print(f"Przewidywana liczba kartek: {predicted_cards}")

Przewidywana liczba goli: 0
Przewidywana liczba rzutów rożnych: 0
Przewidywana liczba fauli: 0
Przewidywana liczba kartek: 0


In [50]:
goals

tensor([[0.0199]], grad_fn=<AddmmBackward0>)

In [44]:
# Przykładowe dane
general_data = pd.DataFrame(np.random.rand(10, 5))  # 10 próbek, 5 cech
h2h_data = pd.DataFrame(np.random.rand(10, 5))  # 10 próbek, 5 cech

# Konwersja na Tensory PyTorch
general_features = torch.tensor(general_data.values, dtype=torch.float32)
h2h_features = torch.tensor(h2h_data.values, dtype=torch.float32)

In [45]:
general_features

tensor([[0.8186, 0.6336, 0.2327, 0.9308, 0.4397],
        [0.5828, 0.7696, 0.5506, 0.0372, 0.1350],
        [0.1965, 0.4906, 0.2309, 0.4378, 0.1039],
        [0.9084, 0.5413, 0.7994, 0.7978, 0.1618],
        [0.2549, 0.5177, 0.3100, 0.5998, 0.5813],
        [0.5358, 0.3724, 0.5937, 0.7571, 0.9037],
        [0.3314, 0.1399, 0.2412, 0.2985, 0.4733],
        [0.0912, 0.2866, 0.0975, 0.5327, 0.4665],
        [0.5683, 0.0579, 0.6679, 0.9305, 0.0301],
        [0.6243, 0.7807, 0.9294, 0.0055, 0.3299]])

In [43]:
# Załóżmy, że mamy DataFrame'y Pandas dla obu rodzajów danych
import pandas as pd
import numpy as np

# Przykładowe dane
general_data = pd.DataFrame(np.random.rand(10, 5))  # 10 próbek, 5 cech
h2h_data = pd.DataFrame(np.random.rand(10, 5))  # 10 próbek, 5 cech

# Konwersja na Tensory PyTorch
general_features = torch.tensor(general_data.values, dtype=torch.float32)
h2h_features = torch.tensor(h2h_data.values, dtype=torch.float32)

# Użycie modelu
model = FootballLSTM(input_dim=5, hidden_dim=100, num_layers=2)
result, goals, corners, fouls, cards = model(general_features, h2h_features)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [4]:
model = torch.nn.LSTM(3, 3)

In [5]:
inputs = [torch.randn(1,3) for _ in range(5)]

In [6]:
inputs

[tensor([[-2.4757, -0.0613,  0.8765]]),
 tensor([[-0.7576,  0.5321,  1.3035]]),
 tensor([[ 1.1223, -0.4409,  0.8695]]),
 tensor([[-0.7415,  0.7655,  0.2143]]),
 tensor([[0.6256, 0.0883, 0.9677]])]

In [16]:
hidden_state = (torch.randn(1,1,3), torch.randn(1,1,3))

In [ ]:
for i in inputs:
    out

In [19]:
for i in inputs:
    out, hidden = model(i.view(1,1,-1), hidden_state)
    print(out)
    print(hidden)

tensor([[[-0.0234, -0.4240,  0.0158]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.0234, -0.4240,  0.0158]]], grad_fn=<StackBackward0>), tensor([[[-0.1086, -0.6122,  0.0251]]], grad_fn=<StackBackward0>))
tensor([[[-0.0343, -0.4233, -0.1624]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.0343, -0.4233, -0.1624]]], grad_fn=<StackBackward0>), tensor([[[-0.1067, -0.6368, -0.3032]]], grad_fn=<StackBackward0>))
tensor([[[-0.1239, -0.2604, -0.1551]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.1239, -0.2604, -0.1551]]], grad_fn=<StackBackward0>), tensor([[[-0.3531, -0.6143, -0.5484]]], grad_fn=<StackBackward0>))
tensor([[[-0.0354, -0.3647, -0.0885]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.0354, -0.3647, -0.0885]]], grad_fn=<StackBackward0>), tensor([[[-0.1265, -0.5377, -0.1604]]], grad_fn=<StackBackward0>))
tensor([[[-0.0843, -0.3082, -0.1784]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.0843, -0.3082, -0.1784]]], grad_fn=<StackBackward0>), tensor([[[-0.2376

In [20]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

In [21]:
inputs

tensor([[[-2.4757, -0.0613,  0.8765]],

        [[-0.7576,  0.5321,  1.3035]],

        [[ 1.1223, -0.4409,  0.8695]],

        [[-0.7415,  0.7655,  0.2143]],

        [[ 0.6256,  0.0883,  0.9677]]])

In [22]:
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state

In [24]:
out, hidden = model(inputs, hidden)

In [25]:
print(out)
print(hidden)

tensor([[[-0.0027,  0.1892,  0.0612]],

        [[-0.0150, -0.0627, -0.1976]],

        [[-0.1869, -0.0908, -0.1664]],

        [[-0.0975, -0.1816, -0.3243]],

        [[-0.1468, -0.1239, -0.2429]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.1468, -0.1239, -0.2429]]], grad_fn=<StackBackward0>), tensor([[[-0.2601, -0.2058, -0.6776]]], grad_fn=<StackBackward0>))


In [27]:
input_size = 1    # The number of variables in your sequence data. 
n_hidden   = 100  # The number of hidden nodes in the LSTM layer.
n_layers   = 2    # The total number of LSTM models layers
out_size   = 1    # The size of the output you desire from your RNN

lstm   = torch.nn.LSTM(input_size, n_hidden, n_layers)
linear = torch.nn.Linear(n_hidden, 1)

In [28]:
data = np.random.randn(1000)

In [30]:
get_batches(data)

NameError: name 'get_batches' is not defined

In [31]:
import numpy as np
import torch

# Przykład szeregu czasowego
data = np.random.randn(1000)  # 1000 punktów danych

# Definicja długości sekwencji
seq_length = 10

# Przygotowanie danych do LSTM
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

# Tworzenie sekwencji
x_data, y_data = create_sequences(data, seq_length)

# Konwersja na tensor PyTorch
x_data = torch.tensor(x_data, dtype=torch.float32).view(-1, seq_length, 1)  # Reshape to (batch_size, seq_length, num_features)
y_data = torch.tensor(y_data, dtype=torch.float32)

In [33]:
def get_batches(x, y, batch_size):
    n_batches = len(x) // batch_size
    x = x[:n_batches * batch_size]
    y = y[:n_batches * batch_size]
    
    for i in range(0, len(x), batch_size):
        yield x[i:i+batch_size], y[i:i+batch_size]

In [35]:
def get_batches(data, batch_size):
    """
    Dzieli dane na partie o określonym rozmiarze.
    :param data: Tensor danych o wymiarach (total_samples, seq_length, num_features)
    :param batch_size: Rozmiar jednej partii danych
    :return: Generator partii danych
    """
    total_samples = data.size(0)
    
    # Przechodzimy przez dane w krokach o wielkości `batch_size`
    for i in range(0, total_samples, batch_size):
        x = data[i:i + batch_size]
        yield x

In [38]:
data = np.random.randn(1000)

In [39]:
get_batches(data, 20)

<generator object get_batches at 0x000001DD92659540>

In [46]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

In [48]:
input

tensor([[[ 0.3402, -1.0346, -1.6351,  0.5458,  1.9475,  0.3342, -1.0398,
          -0.6201,  0.6268,  0.5607],
         [ 0.7123, -1.9538, -0.5583, -0.7741,  1.1485,  0.7548, -0.7387,
           1.1059, -2.3533, -0.1311],
         [-1.3142, -0.7927,  0.6968,  1.5350, -0.0881,  0.0739,  1.1037,
           1.8096, -0.2372, -2.1510]],

        [[-1.7891, -1.1571, -0.3987, -0.4641, -0.2216, -1.7661,  0.1770,
           0.3268, -1.0840, -2.1622],
         [ 0.4804, -0.6694,  0.8168,  0.0583,  0.1769, -0.7708,  1.2248,
          -1.1555, -0.9373,  0.1350],
         [ 0.9253,  0.7556, -0.9968,  1.2333,  0.8377, -0.1685,  3.7096,
          -1.1110, -1.3701, -0.0440]],

        [[ 1.8915, -0.4202,  0.8943,  0.9341, -0.2192,  0.2484, -2.0439,
           0.3170, -0.5367, -0.0198],
         [ 0.9351,  0.0613,  1.0629,  0.3688,  0.3416, -1.2335, -0.3948,
          -2.3961, -0.1503, -0.2130],
         [ 0.4049,  1.4194,  0.4130, -1.0066, -0.6485,  1.3802, -1.7308,
          -0.6369,  0.0708, -1.7277